# Cleansing of US Census Data
In this project, we will be working with a series of spreadsheets obtained from the US Census Bureau.  The data will be imported from multiple spreadsheets, cleansed, and prepared for analysis and visualization using tools in Python.

## Importing Libraries and Data
First, we will import the libraries to use.  This includes Glob to assist in importing the spreadsheets, Pandas and Numpy for data organization and analysis, and Pyplot for data visualization.

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob

# Making the default plot size larger for easier visualization
plt.rcParams['figure.figsize'] = [10, 5]

In [33]:
# The data is spread across 10 CSV files.  We have moved the files to their own 'Data' folder and will use
# glob to iterate through them and read their contents into dataframes

path = 'Data'
files = glob.glob(path + '/*.csv')
df_list = [pd.read_csv(file) for file in files]

# Now combining the individual dataframes into one

df = pd.concat(df_list, axis = 0)

# And inspecting the first few rows and data types

print(df.head())
print(df.dtypes)

   Unnamed: 0           State  TotalPop Hispanic   White   Black Native  \
0           0    Rhode Island   1053661   13.36%  74.33%   5.68%  0.35%   
1           1  South Carolina   4777576    5.06%  62.89%  28.75%  0.29%   
2           2    South Dakota    843190    3.24%  82.50%   1.42%  9.42%   
3           3       Tennessee   6499615    4.72%  73.49%  18.28%  0.23%   
4           4           Texas  26538614   38.05%  44.69%  11.65%  0.26%   

   Asian Pacific       Income            GenderPop  
0  3.25%   0.04%  $59,125.27       510388M_543273F  
1  1.25%   0.05%  $46,296.81     2322409M_2455167F  
2  1.02%   0.04%  $51,805.41       423477M_419713F  
3  1.41%   0.04%  $47,328.08     3167756M_3331859F  
4  3.67%   0.07%  $55,874.52   13171316M_13367298F  
Unnamed: 0     int64
State         object
TotalPop       int64
Hispanic      object
White         object
Black         object
Native        object
Asian         object
Pacific       object
Income        object
GenderPop     object


## Cleaning the Data
From the initial inspection of our imported data, the first things we may notice is that it is out of order and has a column named 'Unnamed: 0' which appears to be a duplicate of the indices.  We will address this by sorting on the 'State' column, removing the 'Unnamed: 0' column, and resetting the indices.

In [34]:
df = df.sort_values(by = 'State').drop(['Unnamed: 0'], axis = 1).reset_index(drop = True)
print(df.head())

        State  TotalPop Hispanic   White   Black  Native   Asian Pacific  \
0     Alabama   4830620    3.75%  61.88%  31.25%   0.45%   1.05%   0.03%   
1      Alaska    733375    5.91%  60.91%   2.85%  16.39%   5.45%   1.06%   
2     Arizona   6641928   29.57%  57.12%   3.85%   4.36%   2.88%   0.17%   
3    Arkansas   2958208    6.22%  71.14%  18.97%   0.52%   1.14%   0.15%   
4  California  38421464   37.29%  40.22%   5.68%   0.41%  13.05%   0.35%   

        Income            GenderPop  
0  $43,296.36     2341093M_2489527F  
1  $70,354.74       384160M_349215F  
2  $54,207.82     3299088M_3342840F  
3  $41,935.63     1451913M_1506295F  
4  $67,264.78   19087135M_19334329F  


Now we can look at cleaning the values themselves.  We'll want to perform analyses on the data which would currently be prevented by the special characters such as dollar signs and percentages.  Our next step will be to remove these special characters and then convert the columns to numerics.

In [35]:
# Removing all commas, dollar signs, and percent signs using regex.

df.replace('[$%,]', '', regex = True, inplace = True)

# Converting relevant columns to numeric.

df[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Income']] = \
    df[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific', 'Income']].apply(pd.to_numeric)

# Converting percentages to their decimal format

df[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']] = \
    round(df[['Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific']] / 100, 4)

print(df.dtypes)

State         object
TotalPop       int64
Hispanic     float64
White        float64
Black        float64
Native       float64
Asian        float64
Pacific      float64
Income       float64
GenderPop     object
dtype: object


One messy column remains which is 'GenderPop'.  We can see from our earlier inspection that this column contains counts of a state's population by gender combined into a single string.  To make this information useful, we'll split it into two additional columns: 'Male' and 'Female'.  These columns will contain the percentage of males and females making up the population of the state.

In [36]:
# First we remove the 'M' and 'F' indicators and split the column on the underscore.

mf_count = list(df.GenderPop.replace('[MF]', '', regex = True).str.split('_'))

# Then we create the new columns using list comprehension

df['Male'] = [i[0] for i in mf_count]
df['Female'] = [i[1] for i in mf_count]

# We can now easily convert the columns to their percentages

df.Male = round(pd.to_numeric(df.Male) / df.TotalPop, 4)
df.Female = round(pd.to_numeric(df.Female) / df.TotalPop, 4)

During this process, we may have noticed that there are several missing (NaN) values in the 'Female' column.  Since we still have the total population and the proportion of males, we can infer the values and fill in the missing fields.

In [37]:
df.Female = df.Female.fillna(1 - df.Male)

Now that our data is clean there is another issue we'll want to address:

In [38]:
print(df.shape)

(60, 12)


We have 60 rows in our data - it would appear we have duplicates.  We can remedy that very simply.

In [39]:
df = df.drop_duplicates().reset_index(drop = True)
print(df.shape)

(51, 12)


Our data is now clean enough for analysis and visualization.

In [40]:
df.head()

,State,TotalPop,Hispanic,White,Black,Native,Asian,Pacific,Income,GenderPop,Male,Female
0,Alabama,4830620,0.0375,0.6188,0.3125,0.0045,0.0105,0.0003,43296.36,2341093M_2489527F,0.4846,0.5154
1,Alaska,733375,0.0591,0.6091,0.0285,0.1639,0.0545,0.0106,70354.74,384160M_349215F,0.5238,0.4762
2,Arizona,6641928,0.2957,0.5712,0.0385,0.0436,0.0288,0.0017,54207.82,3299088M_3342840F,0.4967,0.5033
3,Arkansas,2958208,0.0622,0.7114,0.1897,0.0052,0.0114,0.0015,41935.63,1451913M_1506295F,0.4908,0.5092
4,California,38421464,0.3729,0.4022,0.0568,0.0041,0.1305,0.0035,67264.78,19087135M_19334329F,0.4968,0.5032
